## Función que devuelve la lista de distritos del conjunto de datos del OIJ que
## no coinciden con ningún distrito del conjunto de datos del INEC.

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import trim, lower, col, split

spark = SparkSession.builder \
    .appName("Proyecto SparkSQL") \
    .config("spark.jars.packages", "org.postgresql:postgresql:42.7.4") \
    .getOrCreate()
    
# función que devuelve la lista de distritos del conjunto de datos del OIJ que no coinciden con ningún distrito del INEC
def find_non_matching_districts(df_oij, df_inec):
    """
    Encuentra y devuelve una lista de distritos del conjunto de datos del OIJ que no coinciden
    con ningún distrito del conjunto de datos del INEC.
    
    Args:
        df_oij: DataFrame de Spark que contiene los datos del OIJ
        df_inec: DataFrame de Spark que contiene los datos del INEC
        
    Returns:
        DataFrame con los distritos del OIJ que no tienen coincidencias en el conjunto de datos del INEC
    """
    # validar que los DataFrames no estén vacíos
    if df_oij.rdd.isEmpty() or df_inec.rdd.isEmpty():
        print("Uno o ambos DataFrames están vacíos.")
        return None

    # validar que las columnas existan
    if "Distrito" not in df_oij.columns or "distrito" not in df_inec.columns:
        print("Columnas requeridas no encontradas en los DataFrames.")
        return None

    # para asegurarse de que los distritos estén en minúsculas y sin espacios adicionales
    df_oij_clean = df_oij.withColumn("Distrito", trim(lower(col("Distrito"))))
    df_inec_clean = df_inec.withColumn("distrito", trim(lower(col("distrito"))))

    # realizar un left anti join para encontrar distritos en OIJ que no estén en INEC
    non_matching_districts = df_oij_clean.join(
        df_inec_clean,
        df_oij_clean["Distrito"] == df_inec_clean["distrito"],
        "left_anti"
    ).select("Distrito").distinct()

    return non_matching_districts

# cargar los conjuntos de datos
df_oij = spark.read.csv("C:\\Users\\andco\\Desktop\\spark\\data\\OIJ.csv", header=True, inferSchema=True)
df_inec = spark.read.csv("C:\\Users\\andco\\Desktop\\spark\\data\\inec.csv", header=True, inferSchema=True)

# extraer columna de distrito de INEC
df_inec_clean = df_inec.withColumn(
    'distrito',
    trim(lower(split(col('Provincia, cantón y distrito'), ',').getItem(2)))
)

# aplicar la función para encontrar distritos no coincidentes
non_matching = find_non_matching_districts(df_oij, df_inec_clean)

# mostrar el resultado si hay datos
if non_matching:
    non_matching.show()
else:
    print("No hay distritos no coincidentes o hubo un error en la comparación.")

AnalysisException: [PATH_NOT_FOUND] Path does not exist: file:/C:/Projects/Spark/data/OIJ.csv.

: 